In [1]:
import os
import pandas as pd
import pickle

from scipy.signal import welch, butter, filtfilt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Serialize and Unserialize RAW data

In [2]:
column_names = [
    "Sample Index",
    "EEG Channel 0", "EEG Channel 1", "EEG Channel 2", "EEG Channel 3",
    "EEG Channel 4", "EEG Channel 5", "EEG Channel 6", "EEG Channel 7",
    "Accelerometer X", "Accelerometer Y", "Accelerometer Z",
    "Timestamp", "Timestamp (Formatted)"
]

# Walk through Data files, create dataframes, and pickle them
for root, dirs, files in os.walk('./Data'):
    for file in files:
        if file.endswith(".csv"):
            csv_path = os.path.join(root, file)
            relative_path = os.path.relpath(csv_path, './Data')
            pkl_path = os.path.join('./Data', relative_path).replace(".csv", ".pickle")

            os.makedirs(os.path.dirname(pkl_path), exist_ok=True)

            try:
                df = pd.read_csv(csv_path, sep='\t', header=None)
                # Just grab the first twelve and second to last columns
                df_filtered = df[list(df.columns[:12]) + [df.columns[-2]]].copy()
                # Create a new column with formatted timestamps
                df_filtered["Timestamp (Formatted)"] = pd.to_datetime(df_filtered.iloc[:, -1], unit='s', utc=True)
                # And update the column names
                df_filtered.columns = column_names
                # And cut off the first row
                df_filtered = df_filtered[1:].copy()
                # Finally, pickle the dataframe
                df_filtered.to_pickle(pkl_path)
            except Exception as e:
                print(f"Error processing {csv_path}: {e}")

In [3]:
# From pickles, create one large 'data' dataframe
# Structure: data[participant][category][Optional[task]][Optional[subtask]]
#   ex. data[1][Baselines]
#   ex. data[1][Rhythm][Karate]
#   ex. data[1][Reading][English][In]
data = {}

for root, _, files in os.walk('./Data'):
    for file in files:
        if file.endswith(".pickle") and root != './Data':
            filepath = os.path.join(root, file)
            parts = filepath.split(os.sep)
            df = pd.read_pickle(filepath)

            participant = parts[-2]
            category = parts[2]
            if category != 'Baselines':
                task = parts[3]
                if len(parts) > 6:
                    data.setdefault(participant, {}).setdefault(category, {}).setdefault(task, {})[parts[4]] = df
                else:
                    data.setdefault(participant, {}).setdefault(category, {})[task] = df
            else:
                data.setdefault(participant, {})[category] = df

with open('./Data/all_data_RAW.pickle', 'wb') as fp:
    pickle.dump(data, fp)

# Scale data up
# scale_factor = 4.5 / 24 / (2**23 - 1) * 1e6

# for participant in data:
#     for category in data[participant]:
#         category_data = data[participant][category]

#         if isinstance(category_data, dict):
#             for task in category_data:
#                 task_data = category_data[task]

#                 if isinstance(task_data, dict):
#                     for subtask in task_data:
#                         df = task_data[subtask]
#                         for ch in range(8):
#                             col = f"EEG Channel {ch}"
#                             df[col] = df[col] * scale_factor
#                 else:
#                     df = task_data
#                     for ch in range(8):
#                         col = f"EEG Channel {ch}"
#                         df[col] = df[col] * scale_factor
#         else:
#             df = category_data
#             for ch in range(8):
#                 col = f"EEG Channel {ch}"
#                 df[col] = df[col] * scale_factor

# with open('./Data/all_data_SCALED.pickle', 'ab') as fp:
#     pickle.dump(data, fp)
# with open('./Data/all_data_RAW.pickle', 'rb') as fp:
#     raw_data = pickle.load(fp)

# Serialize and Unserialize FILTERED data

In [4]:
def plot_data(data, title, path):
    fig, axs = plt.subplots(8, 1, figsize=(12, 10), sharex=True)
    fig.suptitle(title, fontsize=16)

    colors = ['#afb6be', '#7c4b8d', '#36579e', '#317159', '#ddb20d', '#fd5e34', '#e0382d', '#a25231']
    for i, channel in enumerate([f"EEG Channel {i}" for i in range(8)]):
        axs[i].plot(data[channel], linewidth=1, color=colors[i])
        axs[i].set_ylabel(f"Channel {i}")
        axs[i].set_yticklabels([])
        axs[i].set_yticks([])
        axs[i].set_xticks([])

    plt.tight_layout(pad=0.5, h_pad=0)
    plt.subplots_adjust(top=0.95)
    plt.savefig(path)
    plt.close()


b, a = butter(2, (0.1, 80.1), 'bandpass', fs=250)
def filter_df(df):
    for ch in [f"EEG Channel {i}" for i in range(8)]:
        df[ch] = filtfilt(b, a, df[ch])
    return df


data_filtered = {}
for participant in data:
    data_filtered[participant] = {}
    for task in data[participant]:
        if task == 'Baselines':
            df = data[participant][task]
            df_filtered = filter_df(df.copy())
            data_filtered[participant][task] = df_filtered
            plot_data(df_filtered, f'Participant no. {participant}: {task}', f'./Data/{task}/{participant}/{participant}_filtered.png')
        else:
            data_filtered[participant][task] = {}
            for subtask in data[participant][task]:
                if task == 'Reading':
                    data_filtered[participant][task][subtask] = {}
                    for direction in data[participant][task][subtask]:
                        df = data[participant][task][subtask][direction]
                        df_filtered = filter_df(df.copy())
                        data_filtered[participant][task][subtask][direction] = df_filtered
                        plot_data(df_filtered, f'Participant no. {participant}: {task}, {subtask} ({direction})', f'./Data/{task}/{subtask}/{direction}/{participant}/{participant}_filtered.png')
                else:
                    df = data[participant][task][subtask]
                    df_filtered = filter_df(df.copy())
                    data_filtered[participant][task][subtask] = df_filtered
                    plot_data(df_filtered, f'Participant no. {participant}: {task}, {subtask}', f'./Data/{task}/{subtask}/{participant}/{participant}_filtered.png')

with open('./Data/all_data_FILTERED.pickle', 'wb') as fp:
    pickle.dump(data_filtered, fp)

In [5]:
with open('./Data/all_data_FILTERED.pickle', 'rb') as fp:
    data_filtered = pickle.load(fp)

# Serialize and Unserialize PSD bandpower data

In [6]:
bands = [
    (0, 4, "Delta", "#e0382d"),
    (4, 8, "Theta", "#ddb20d"),
    (8, 10, "Alpha", "#317159"),
    (10, 13, "Mu", "#358a9e"),
    (13, 20, "Beta I", "#36579e"),
    (20, 50, "Beta II", "#48359e"),
    (50, 125, "Gamma", "#7c4b8d"),
]
def plot_welch_PSD(df, title, path):
    """ Welch Power Spectral Density """
    channels = [f"EEG Channel {i}" for i in range(8)]
    bandpowers = {ch: {} for ch in channels}
    fig, axs = plt.subplots(4, 2, figsize=(14, 10), sharex=True, sharey=True)
    axs = axs.flatten()

    for i, channel in enumerate(channels):
        freqs, power = welch(df[channel], fs=250, nperseg=1024)

        axs[i].semilogy(freqs, power, color='white')
        axs[i].set_title(f"{channel} PSD")
        axs[i].set_xlim([0, 80])
        axs[i].set_xticks([])
        axs[i].set_yticklabels([])
        axs[i].set_yticks([])
        axs[i].grid(True)
        
        for j, (low, high, label, color) in enumerate(bands):
            if high > 125:
                continue
            axs[i].axvspan(low, high, color=color, alpha=0.75, label=label)

            band_mask = (freqs >= low) & (freqs <= high)
            if np.any(band_mask):
                band_freqs = freqs[band_mask]
                band_power = power[band_mask]
                peak_idx = np.argmax(band_power)
                peak_freq = band_freqs[peak_idx]
                peak_pow = band_power[peak_idx]

                bandpowers[channel][label] = np.trapezoid(power[band_mask], freqs[band_mask])

                xoffset = 5 if j == 6 or j == 0 else 0
                yoffset = 10 if j == 0 else 0.001 if j % 2 == 1 or j == 6 else 30

                axs[i].annotate(f"{peak_freq:.1f} Hz",
                                xy=(peak_freq, peak_pow),
                                xytext=(peak_freq + xoffset, peak_pow * yoffset),
                                arrowprops=dict(arrowstyle="->", lw=0.8),
                                fontsize=8, ha='center')

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, ncol=7, fontsize='small', bbox_to_anchor=(0.7225, 0.9575))
    plt.suptitle(f"Welch Power Spectral Density - {title}", fontsize=16)
    fig.text(0.5, 0.01, 'Frequency (Hz)', ha='center', fontsize=12)
    fig.text(0.01, 0.5, 'Power (μV²/Hz)', va='center', rotation='vertical', fontsize=12)
    plt.tight_layout(pad=0.5, w_pad=1)
    plt.subplots_adjust(top=0.89, left=0.03, bottom=0.035)
    plt.savefig(path)
    plt.close()

    return bandpowers

data_bandpowers = {}
for participant in data_filtered:
    data_bandpowers[participant] = {}
    for task in data_filtered[participant]:
        if task == 'Baselines':
            df = data_filtered[participant][task].copy()
            data_bandpowers[participant][task] = plot_welch_PSD(df, f'Participant no. {participant}: {task}', f'./Data/{task}/{participant}/{participant}_PSD.png')
        else:
            data_bandpowers[participant][task] = {}
            for subtask in data_filtered[participant][task]:
                if task == 'Reading':
                    data_bandpowers[participant][task][subtask] = {}
                    for direction in data_filtered[participant][task][subtask]:
                        df = data_filtered[participant][task][subtask][direction].copy()
                        data_bandpowers[participant][task][subtask][direction] = plot_welch_PSD(df, f'Participant no. {participant}: {task}, {subtask} ({direction})', f'./Data/{task}/{subtask}/{direction}/{participant}/{participant}_PSD.png')
                else:
                    df = data_filtered[participant][task][subtask].copy()
                    data_bandpowers[participant][task][subtask] = plot_welch_PSD(df, f'Participant no. {participant}: {task}, {subtask}', f'./Data/{task}/{subtask}/{participant}/{participant}_PSD.png')

with open('./Data/all_data_BANDPOWERS.pickle', 'wb') as fp:
    pickle.dump(data_bandpowers, fp)

In [7]:
with open('./Data/all_data_BANDPOWERS.pickle', 'rb') as fp:
    data_bandpowers = pickle.load(fp)

# Other stuff